### 교차 검증과 그리드 서치

##### 검증 세트

In [40]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [41]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [42]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [43]:
sub_input, val_input, sub_target, val_target, = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [44]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [45]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


##### 교차 검증

In [46]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00700116, 0.00400352, 0.00501585, 0.00503588, 0.00510359]),
 'score_time': array([0.00099683, 0.0010004 , 0.00099182, 0.0009532 , 0.0009737 ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [47]:
import numpy as np

np.mean(scores['test_score'])

0.855300214703487

In [48]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [49]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

##### 하이퍼파라미터 튜닝

In [57]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005,]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [51]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [52]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [53]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

In [61]:
params = {
        'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
        'max_depth': range(5, 20, 1),
        'min_samples_split': range(2, 100, 10)
}

In [63]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [64]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

##### 랜덤서치

In [65]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([6, 5, 6, 4, 1, 2, 7, 2, 7, 5], dtype=int64)

In [66]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 91, 111,  85,  98, 104,  85, 118, 119,  94,  95], dtype=int64))

In [67]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.8836164 , 0.6598526 , 0.1426604 , 0.75762123, 0.29423812,
       0.45079089, 0.25137009, 0.30897352, 0.68501893, 0.96794172])

In [68]:
params = {
        'min_impurity_decrease': uniform(0.0001, 0.001),
        'max_depth': randint(20, 50),
        'min_samples_split': randint(2, 25),
        'min_samples_leaf': randint(1, 25),
}

In [70]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [71]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [72]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.86